In [1]:
!pip install -q mne loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/INL')

Mounted at /content/drive


In [3]:
import torch
import numpy as np
import mne
from loguru import logger
from transform import MNEFilter
from torch_dataset import TimeseriesDataset
import torchaudio
import torchvision
import torchvision.transforms as T

In [4]:
SFREQ = 585
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cpu


In [5]:
def load_x_data(filepath = '/content/drive/MyDrive/Colab Notebooks/INL/processed_data/X_train2.npy'):
  np.load('/content/drive/MyDrive/Colab Notebooks/INL/processed_data/Train_data.npy', allow_pickle = True)
  x_data =  np.load(filepath)
  return x_data

def load_y_data(filepath = '/content/drive/MyDrive/Colab Notebooks/INL/processed_data/label2.npy'):
  y_data =  np.load(filepath)
  return y_data


In [25]:
def load_ecogs():
    signals = np.load('/content/drive/MyDrive/Colab Notebooks/INL/processed_data/Train_data.npy', allow_pickle = True)
    channels = np.load('/content/drive/MyDrive/Colab Notebooks/INL/processed_data/Ch_train.npy', allow_pickle = True)
    ecog_signals = []
    for i in range(len(signals)):
      signal = signals[i]
      ch_names = channels[i]
      ecog_ch_names = [ch for ch in ch_names if "ecog" in ch]
      ecog_signal = signal[[ch_names.index(ch) for ch in ecog_ch_names]]
      ecog_signals.append(ecog_signal)

    ecog_signals = [signal for signal in ecog_signals if signal.shape[1] != 0] #remove empty signals
    ecog_signals = np.hstack(ecog_signals) if ecog_signals else np.array([])

    return ecog_signals


def load_states(return_ch_names=False):
    signals = np.load('/content/drive/MyDrive/Colab Notebooks/INL/processed_data/Train_data.npy', allow_pickle = True)
    channels = np.load('/content/drive/MyDrive/Colab Notebooks/INL/processed_data/Ch_train.npy', allow_pickle = True)
    state_signals = []
    if return_ch_names:
      state_ch_names = []
    for i in range(len(signals)):
      signal = signals[i]
      ch_names = channels[i]
      state_ch_names = [ch for ch in ch_names if "state__" in ch]
      state_signal = signal[[ch_names.index(ch) for ch in state_ch_names]]
      state_signals.append(state_signal)
      if return_ch_names:
        state_ch_names.append(state_ch_names)

    state_signals = [signal for signal in state_signals if signal.shape[1] != 0] #remove empty signals
    state_signals = np.hstack(state_signals) if state_signals else np.array([])
    if return_ch_names:
        return state_signals, state_ch_names
    return state_signals

In [22]:
ecog_signals = load_ecogs()

In [24]:
ecog_signals.shape

(32, 21352159)

In [29]:
#signals = np.load('/content/drive/MyDrive/Colab Notebooks/INL/processed_data/Train_data.npy', allow_pickle = True)
channels = np.load('/content/drive/MyDrive/Colab Notebooks/INL/processed_data/Ch_train.npy', allow_pickle = True)

In [30]:
channels[10]

['add_ch_0',
 'add_ch_1',
 'add_ch_2',
 'add_ch_3',
 'add_ch_4',
 'add_ch_5',
 'add_ch_6',
 'add_ch_7',
 'add_ch_8',
 'is_updating',
 'states',
 'state__idle',
 'state__shoulder__flexion',
 'state__shoulder__extension',
 'state__shoulder__adduction',
 'state__shoulder__abduction',
 'state__shoulder__internal_rotation',
 'state__shoulder__external_rotation',
 'state__elbow__flexion',
 'state__elbow__extension',
 'state__wrist__pronation',
 'state__wrist__supination',
 'state__hand__open',
 'state__hand__close',
 'ecog_0',
 'ecog_1',
 'ecog_2',
 'ecog_3',
 'ecog_4',
 'ecog_5',
 'ecog_6',
 'ecog_7',
 'ecog_8',
 'ecog_9',
 'ecog_10',
 'ecog_11',
 'ecog_12',
 'ecog_13',
 'ecog_14',
 'ecog_15',
 'ecog_16',
 'ecog_17',
 'ecog_18',
 'ecog_19',
 'ecog_20',
 'ecog_21',
 'ecog_22',
 'ecog_23',
 'ecog_24',
 'ecog_25',
 'ecog_26',
 'ecog_27',
 'ecog_28',
 'ecog_29',
 'ecog_30',
 'ecog_31']

In [26]:
state_signals = load_states()

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 13 and the array at index 10 has size 0

In [ ]:
state_signals.shape

In [6]:
x_data = load_x_data()
x_data.shape

(11730, 31, 1770)

In [9]:
y_data = load_y_data()
print(y_data.shape)

(11730,)


In [8]:
y_data.shape

(11730,)

In [ ]:
win_size = 585
win_step = 59
n_fft = win_size
hop_length = 10
filepath_x = '/content/drive/MyDrive/Colab Notebooks/INL/processed_data/X_train2.npy'
filepath_y = '/content/drive/MyDrive/Colab Notebooks/INL/processed_data/label2.npy'

dataset = TimeseriesDataset(
    load_x_func=load_x_data,
    load_x_args={"filepath": filepath_x},
    n_samples_step=win_step,
    n_samples_win=win_size,
    x_preprocess=T.Compose(
        [
            MNEFilter(sfreq=SFREQ, l_freq=1, h_freq=200, notch_freqs=np.arange(50, 201, 50), apply_car=True),
            torchvision.transforms.ToTensor(),
        ],
    ),
    y_preprocess=T.Compose(
        [],
    ),
    x_transform=T.Compose(
        [
            # we first push to GPU, then apply transforms
            lambda x: x.to(DEVICE),
            # needs to be adjusted to the desired output size (FREQ_BINS, TIME_BINS)
            torchaudio.transforms.Spectrogram(
                n_fft=n_fft,
                win_length=n_fft,
                hop_length=hop_length,
                center=True,
                window_fn=lambda x: torch.hann_window(x).to(DEVICE),
            ),
            lambda x: x.squeeze(0).float(),
        ]
    ),
    y_transform=T.Compose(
        [
            lambda x: torch.tensor(x).to(DEVICE),
            # for the sake of this example, we take the last state value
            lambda x: x[..., -1].flatten().float(),
        ]
    ),
    precompute=True,
)

(31, 1770)
Creating RawArray with float64 data, n_channels=31, n_times=1770
    Range : 0 ... 1769 =      0.000 ...     3.024 secs
Ready.
ECoG channel type selected for re-referencing
Applying average reference.
Applying a custom ('ECoG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3861 samples (6.600 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passba

/content/drive/MyDrive/Colab Notebooks/INL/transform.py:64: RuntimeWarning: filter_length (3861) is longer than the signal (1770), distortion is likely. Reduce filter length or filter a longer signal.
  raw.notch_filter(freqs=self.notch_freqs, notch_widths=2, fir_design="firwin")
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
/content/drive/MyDrive/Colab Notebooks/INL/transform.py:67: RuntimeWarning: filter_length (1931) is longer than the signal (1770), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(l_freq=self.l_freq, h_freq=self.h_freq, fir_design="firwin")
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
2024-11-05 20:22:04.355 | INFO     | torch_dataset:__init__:105 - Precomputing dataset (could take a while)
100%|██████████| 21/21 [00:00<00:00, 73.74it/s]


In [ ]:
type(dataset[20])

torch.Tensor

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=False)

In [ ]:
X = next(iter(dataloader))

In [ ]:
X.shape

torch.Size([8, 31, 293, 59])